## K-Fold Cross-Validation

### Завдання 2

Проведіть крос-валідацію для моделі, побудованої на основі датафрейму **petrol_consumption.csv** з попереднього завдання. Обгрунтуйте обраний метод крос-валідації

### Розв'язок

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn import linear_model, tree, ensemble

## Load Dataset

In [19]:
train_data = pd.read_csv('petrol_consumption.csv')
train_data.head()

,Petrol_tax,Average_income,Paved_Highways,Population_Driver_licence(%),Petrol_Consumption
0,9.0,3571,1976,0.525,541
1,9.0,4092,1250,0.572,524
2,9.0,3865,1586,0.580,561
3,7.5,4870,2351,0.529,414
4,8.0,4399,431,0.544,410


In [20]:

train_data.dropna(axis=0, subset=['Petrol_Consumption'], inplace=True)

y = train_data.Petrol_Consumption          
X = train_data.drop(['Petrol_Consumption'], axis=1)

print("Shape of input data: {} and shape of target variable: {}".format(X.shape, y.shape))

pd.concat([X, y], axis=1).head()

Shape of input data: (48, 4) and shape of target variable: (48,)


,Petrol_tax,Average_income,Paved_Highways,Population_Driver_licence(%),Petrol_Consumption
0,9.0,3571,1976,0.525,541
1,9.0,4092,1250,0.572,524
2,9.0,3865,1586,0.580,561
3,7.5,4870,2351,0.529,414
4,8.0,4399,431,0.544,410


## Understanding the Data

Остаточний набір даних містить 4 функції і 48  прикладів. Ми маємо передбачити, які витрати бензину, виходячи з даних про податок на бензин, середній дохід, автомагістралі та наявність водійських ліцензій у населення.

## Model Score Using KFold

In [21]:
kf =KFold(n_splits=5, shuffle=True, random_state=42)

cnt = 1
for train_index, test_index in kf.split(X, y):
    print(f'Fold:{cnt}, Train set: {len(train_index)}, Test set:{len(test_index)}')
    cnt+=1


Fold:1, Train set: 38, Test set:10
Fold:2, Train set: 38, Test set:10
Fold:3, Train set: 38, Test set:10
Fold:4, Train set: 39, Test set:9
Fold:5, Train set: 39, Test set:9


In [22]:
count = 1
for train_index, test_index in kf.split(X):
    print(f'Fold {count} :', train_index, '\nTest:', test_index)
    count += 1

Fold 1 : [ 0  1  2  3  5  6  7  8  9 10 11 13 14 15 16 17 18 20 21 22 23 28 29 30
 31 32 33 34 35 36 38 39 41 42 44 45 46 47] 
Test: [ 4 12 19 24 25 26 27 37 40 43]
Fold 2 : [ 0  1  2  4  5  7 10 11 12 14 16 18 19 20 21 22 23 24 25 26 27 28 29 30
 31 32 34 35 36 37 38 40 41 42 43 44 46 47] 
Test: [ 3  6  8  9 13 15 17 33 39 45]
Fold 3 : [ 1  2  3  4  6  7  8  9 10 12 13 14 15 17 18 19 20 21 22 23 24 25 26 27
 28 33 35 36 37 38 39 40 41 42 43 44 45 47] 
Test: [ 0  5 11 16 29 30 31 32 34 46]
Fold 4 : [ 0  3  4  5  6  7  8  9 11 12 13 14 15 16 17 18 19 20 22 24 25 26 27 28
 29 30 31 32 33 34 37 38 39 40 42 43 45 46 47] 
Test: [ 1  2 10 21 23 35 36 41 44]
Fold 5 : [ 0  1  2  3  4  5  6  8  9 10 11 12 13 15 16 17 19 21 23 24 25 26 27 29
 30 31 32 33 34 35 36 37 39 40 41 43 44 45 46] 
Test: [ 7 14 18 20 22 28 38 42 47]


In [23]:
def rmse(score):
    rmse = np.sqrt(-score)
    print(f'rmse= {rmse}')

### Using Linear Regression

In [24]:
score = cross_val_score(linear_model.LinearRegression(), X, y, cv = kf, scoring="neg_mean_squared_error")
print(f'Scores for each fold: {score}')
rmse(score.mean())

Scores for each fold: [ -4083.25587175 -10468.85637678  -2453.65941292  -3853.05818469
  -3173.21607154]
rmse= 69.32827117082202


### Using Decision Tree Regressor

In [25]:
score = cross_val_score(tree.DecisionTreeRegressor(random_state= 42), X, y, cv = kf, scoring="neg_mean_squared_error")
print(f'Scores for each fold: {score}')
rmse(score.mean())

Scores for each fold: [-17347.7        -17412.3         -6389.6         -7350.66666667
 -13087.22222222]
rmse= 110.98422310300585


### Using Random Forest Regressor

In [26]:
score = cross_val_score(ensemble.RandomForestRegressor(random_state= 42), X, y, cv= kf, scoring="neg_mean_squared_error")
print(f'Scores for each fold are: {score}')
rmse(score.mean())

Scores for each fold are: [ -7539.12317    -12797.03298     -3600.52778     -2946.84586667
  -6053.55496667]
rmse= 81.16290379641839


## Model Tuning using KFold

### Decision Tree Regressor Tuning

In [27]:
max_depth = [1,2,3,4,5,6,7,8,9,10]

for val in max_depth:
    score = cross_val_score(tree.DecisionTreeRegressor(max_depth= val, random_state= 42), X, y, cv = kf, scoring="neg_mean_squared_error")
    print(f'For max depth: {val}')
    rmse(score.mean())

For max depth: 1
rmse= 96.0448885906034
For max depth: 2
rmse= 109.72146653302416
For max depth: 3
rmse= 107.34027059636371
For max depth: 4
rmse= 105.22825041902301
For max depth: 5
rmse= 105.33869350854975
For max depth: 6
rmse= 110.45609896869759
For max depth: 7
rmse= 122.2700794790061
For max depth: 8
rmse= 123.53311521837193
For max depth: 9
rmse= 110.98573481908984
For max depth: 10
rmse= 110.97151786732384


### Random Forest Regressor Tuning

In [28]:
estimators = [50, 100, 150, 200, 250, 300, 350]

for count in estimators:
    score = cross_val_score(ensemble.RandomForestRegressor(n_estimators= count, random_state= 42), X, y, cv= kf, scoring="neg_mean_squared_error")
    print(f'For estimators: {count}')
    rmse(score.mean())

For estimators: 50
rmse= 82.88561863730577
For estimators: 100
rmse= 81.16290379641839
For estimators: 150
rmse= 81.10830165180077
For estimators: 200
rmse= 80.14954668756536
For estimators: 250
rmse= 79.78893146897006
For estimators: 300
rmse= 80.24292820162017
For estimators: 350
rmse= 80.61728010539575


## Висновок

**Вважаю, що найкращий результат і в цьому завданні показує модель лінійної регресії. Хоча моделі випадкових лісів і древовидної регресії вдалося трохи покращити за допомогою тренування.**

**Для тренування і оцінювання моделей обрано К-кратну валідацію, як найпоширеніший метод тренування моделей, та дає досить точний результат.**

**Стратифіковану крос-валідацію застосувати в даному випадку не можна, оскільки предиктор не розподілений на певні класи.
Я це перевірила експериментальним шляхом, підставивши значення у код з лекції. всі результата дорівнювали нулю.**